In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
df = pd.read_csv('../input/train.csv', parse_dates=[0])
test = pd.read_csv('../input/test.csv', parse_dates=[0])

In [ ]:
df.rename(columns={'count':'rentals'},inplace=True)

In [ ]:
df = df.append(test, sort=False)

In [ ]:
for col in ['rentals','registered','casual']:
    df[col] = np.log(df[col]+1)

In [ ]:
df['year'] = df.datetime.dt.year # df['datetime'].dt.year
df['month'] = df.datetime.dt.month
df['day'] = df.datetime.dt.day
df['dayofweek'] = df.datetime.dt.dayofweek
df['hour'] = df['datetime'].dt.hour

In [ ]:
df.set_index('datetime', inplace=True)

In [ ]:
df.head()

In [ ]:
df.resample('30min')['temp'].mean().fillna(method='ffill')

In [ ]:
df.sort_index(inplace=True)

In [ ]:
df.reset_index(inplace=True)

In [ ]:
df.head()

In [ ]:
df['rolling_temp'] = df['temp'].rolling(4, min_periods=1).mean()

In [ ]:
# Separando os dataframes
test = df[df['rentals'].isnull()]
df = df[~df['rentals'].isnull()]

In [ ]:
df.shape, test.shape

In [ ]:
# Selecionar as colunas a serem usadas no treinamento e validação
# Lista das colunas não usadas
removed_cols = ['rentals', 'casual','registered','datetime']

# Separando as colunas a serem usadas no treino
feats = [c for c in df.columns if c not in removed_cols]

In [ ]:
from sklearn.model_selection import train_test_split

train, valid = train_test_split(df, random_state=42)

In [ ]:
train.shape, valid.shape

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(random_state=42, n_estimators=100,n_jobs=-1)

In [ ]:
rf.fit(train[feats],train['rentals'])

In [ ]:
# Fazer as previsões
preds = rf.predict(valid[feats])

In [ ]:
from sklearn.metrics import mean_squared_error

mean_squared_error(valid['rentals'],preds)**(1/2)

In [ ]:
# Preparando os dados para o kaggle

# Criando as previsões para os dados de teste

preds_test = rf.predict(test[feats])

In [ ]:
# Adicionar as previsões ao dataframe
test['count'] = np.exp(preds_test)-1


In [ ]:
# Salvando o arquivo pro kaggle
test[['datetime','count']].to_csv('rf1.csv', index=False)


In [ ]:
test[['datetime','count']]

In [ ]:
train, valid = df[df['day'] <= 15], df.query('day > 15')

In [ ]:
train.shape, valid.shape

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(random_state=42, n_estimators=100,n_jobs=-1)

In [ ]:
rf.fit(train[feats],train['rentals'])

In [ ]:
# Fazer as previsões
preds = rf.predict(valid[feats])

In [ ]:
from sklearn.metrics import mean_squared_error

mean_squared_error(valid['rentals'],preds)**(1/2)

In [ ]:
# Preparando os dados para o kaggle

# Criando as previsões para os dados de teste

preds_test = rf.predict(test[feats])

In [ ]:
# Adicionar as previsões ao dataframe
test['count'] = np.exp(preds_test)-1


In [ ]:
# Salvando o arquivo pro kaggle
test[['datetime','count']].to_csv('rf2.csv', index=False)


In [ ]:
test[['datetime','count']]

In [ ]:
####

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(random_state=42, n_estimators=100,n_jobs=-1,oob_score=True)

In [ ]:
rf.fit(df[feats],df['rentals'])
rf.oob_score_

In [ ]:
# Fazer as previsões
preds = rf.oob_prediction_

In [ ]:
from sklearn.metrics import mean_squared_error

mean_squared_error(df['rentals'],preds)**(1/2)

In [ ]:
# Preparando os dados para o kaggle

# Criando as previsões para os dados de teste

preds_test = rf.predict(test[feats])

In [ ]:
# Adicionar as previsões ao dataframe
test['count'] = np.exp(preds_test)-1


In [ ]:
# Salvando o arquivo pro kaggle
test[['datetime','count']].to_csv('rf3.csv', index=False)


In [ ]:
test[['datetime','count']]

In [ ]:
 def cv(df, test, feats, y_name, k=5):
        score, preds, fis = [], [], []
        chunk = df.shape[0] // k
        
        for i in range(k):
            if i+1 < k:
                valid = df.iloc[i*chunk: (i+1)*chunk]
                train = df.iloc[:i*chunk].append(df.iloc[(i+1)*chunk:])
            else:
                valid: df.iloc[i*chunk:]
                train: df.iloc[:i*chunk]
                    
            rf = RandomForestRegressor(random_state=42, n_jobs=-1, n_estimators=100)
            rf.fit(train[feats],train[y_name])
            
            score.append(mean_squared_error(valid[y_name],rf.predict(valid[feats])) ** (1/2))
            
            preds.append(rf.predict(test[feats]))
            
            fis.append(rf.feature_importances_)
            
            print(i, 'OK')
        return score, preds, fis

In [ ]:
score, preds, fis = cv(df, test, feats, 'rentals')

In [ ]:
score

In [ ]:
pd.Series(score).mean()


In [ ]:
test['count'] = np.exp(pd.DataFrame(preds).mean())-1
test[['datetime','count']].to_csv('rf4.csv', index=False)


In [ ]:
pd.DataFrame(fis, columns=feats).mean().sort_values().plot.barh()